## 1. Data Mining

In [1]:
# 1. Import libraries
import pandas as pd
import numpy as np

# 2. Load dataset
df = pd.read_csv("mall_customers.csv")

# 3. Simulate messy data (optional for realism)
df.loc[5, 'Annual Income (k$)'] = np.nan
df.loc[20, 'Age'] = np.nan
df = pd.concat([df, df.iloc[[3]]], ignore_index=True)  # duplicate row

# 4. Preview
print("First 5 rows:")
print(df.head())

print("\nDataset Info:")
print(df.info())

print(f"\nDataset Shape: {df.shape}")

First 5 rows:
   CustomerID   Genre   Age  Annual Income (k$)  Spending Score (1-100)
0           1    Male  19.0                15.0                      39
1           2    Male  21.0                15.0                      81
2           3  Female  20.0                16.0                       6
3           4  Female  23.0                16.0                      77
4           5  Female  31.0                17.0                      40

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CustomerID              201 non-null    int64  
 1   Genre                   201 non-null    object 
 2   Age                     200 non-null    float64
 3   Annual Income (k$)      200 non-null    float64
 4   Spending Score (1-100)  201 non-null    int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 8.0+ KB
No